In [ ]:
from src.data_processing import load_merged_data
from src.visualization import plot_correlation_heatmap

In [ ]:
merged_df = load_merged_data()

In [ ]:
import numpy as np

lags = range(-5, 6)
correlations = []

for lag in lags:
    corr = merged_df['rating_score'].corr(merged_df['price_change'].shift(-lag))
    correlations.append(corr)

plt.figure(figsize=(10,5))
plt.plot(lags, correlations, marker='o')
plt.axvline(0, color='red', linestyle='--')
plt.title('Sentiment-Price Correlation at Different Lags')
plt.xlabel('Days After Rating')
plt.ylabel('Pearson Correlation')
plt.grid(True)
plt.tight_layout()
plt.savefig('../figures/lag_correlation.png')
plt.show()